## CFA Data Exploratory Analysis

### 10/28
### Tina

In [38]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import os
from dotenv import load_dotenv
import json
from datetime import date, datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
import seaborn as sns



Load environment variables

In [39]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

Connect to the database

In [40]:
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("Connected")
except Exception as e:
    print("Connection failed:", e)
    exit()

Connected


Load all public tables into DataFrames

In [41]:
tables_dict = {}

cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name;
""")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    query = sql.SQL("SELECT * FROM {} LIMIT 1000").format(sql.Identifier(table_name))
    try:
        df = pd.read_sql_query(query.as_string(conn), conn)
        tables_dict[table_name] = df
        print(f"Loaded {table_name} ({df.shape[0]} rows, {df.shape[1]} columns)")
    except Exception as e:
        print(f"Could not load {table_name}: {e}")


Loaded client_visit (1000 rows, 50 columns)
Loaded inventory_group (9 rows, 10 columns)
Loaded inventory_group_item (17 rows, 8 columns)
Loaded inventory_group_location_manifest (0 rows, 8 columns)
Loaded inventory_item (716 rows, 31 columns)


C:\Users\TinTin\AppData\Local\Temp\ipykernel_2164\1313306925.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query.as_string(conn), conn)
C:\Users\TinTin\AppData\Local\Temp\ipykernel_2164\1313306925.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query.as_string(conn), conn)


Loaded inventory_item_code (158 rows, 9 columns)
Loaded inventory_item_location_manifest (1000 rows, 8 columns)
Loaded inventory_item_tag (11 rows, 16 columns)
Loaded inventory_item_type (26 rows, 12 columns)
Loaded inventory_items_inventory_item_tags (116 rows, 2 columns)
Loaded inventory_receive (1000 rows, 16 columns)
Loaded inventory_receive_group (0 rows, 9 columns)
Loaded inventory_receive_item (1000 rows, 14 columns)
Loaded inventory_source (239 rows, 21 columns)
Loaded location (10 rows, 22 columns)
Loaded visit_answer (1000 rows, 15 columns)
Loaded visit_budget_item (1000 rows, 9 columns)
Loaded visit_item (1000 rows, 18 columns)
Loaded visit_member (1000 rows, 18 columns)


C:\Users\TinTin\AppData\Local\Temp\ipykernel_2164\1313306925.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query.as_string(conn), conn)


Basic info per table

In [ ]:
import json
import numpy as np
import pandas as pd
from datetime import datetime, date

# -----------------------------
# Utility: make objects JSON safe
# -----------------------------
def make_json_safe(obj):
    """Recursively convert non-JSON-serializable objects."""
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    elif isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    elif isinstance(obj, dict):
        return {str(k): make_json_safe(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_json_safe(i) for i in obj]
    else:
        return obj

# -----------------------------
# Main summary collector
# -----------------------------
summary = {}

for name, df in tables_dict.items():
    # --- Data type & cardinality summary ---
    dtype_info = df.dtypes.astype(str)
    nunique = df.nunique(dropna=True)
    dtype_summary = pd.DataFrame({
        "dtype": dtype_info,
        "unique_values": nunique,
        "missing_%": (df.isna().sum() / len(df) * 100).round(2)
    }).sort_values(by="unique_values", ascending=False)
    
    # --- Table-level summary ---
    table_summary = {
        "shape": {"rows": int(df.shape[0]), "columns": int(df.shape[1])},
        "columns": list(df.columns),
        "dtype_summary": make_json_safe(dtype_summary.to_dict(orient="index")),
        "missing_values_percent": make_json_safe((df.isna().sum() / len(df) * 100).round(2).to_dict()),
        "numeric_summary": {},
        "categorical_summary": {}
    }

    # --- Numeric summary ---
    num_cols = df.select_dtypes(include='number').columns
    if len(num_cols) > 0:
        desc = df[num_cols].describe().T
        table_summary["numeric_summary"] = make_json_safe(desc.to_dict())

    # --- Categorical summary ---
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols:
        counts = df[col].value_counts().head(5).to_dict()
        table_summary["categorical_summary"][col] = make_json_safe(counts)

    summary[name] = table_summary

    # --- Clean print to console ---
    print("\n" + "="*60)
    print(f"TABLE: {name.upper()}")
    print("="*60)
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")

    print("Top Columns by Uniqueness:")
    print(dtype_summary.head(10).to_string())
    print("\nMissing Values (%):")
    for col, pct in table_summary["missing_values_percent"].items():
        if pct > 0:
            print(f"  {col:<30} {pct:.2f}%")

    print("\nNumeric Columns:", list(num_cols))
    print("Categorical Columns:", list(cat_cols))


output_json_path = "table_summary.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(make_json_safe(summary), f, indent=4, ensure_ascii=False)



daily activity trends

In [43]:
output_dir = "daily_activity_trends"
os.makedirs(output_dir, exist_ok=True)

for name, df in tables_dict.items():
    time_cols = [c for c in df.columns if "created_at" in c or "datetime" in c]
    
    for col in time_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        ts = df[col].dropna().dt.date.value_counts().sort_index()

        if len(ts) > 0:
            # --- Plot ---
            plt.figure(figsize=(12, 6))
            ts.plot(kind='line', marker='o', linewidth=2, alpha=0.7)
            plt.title(f"Activity Over Time — {name}.{col}", fontsize=16, fontweight='bold')
            plt.xlabel("Date", fontsize=12)
            plt.ylabel("Number of Records", fontsize=12)
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.tight_layout()

            # --- Save plot ---
            safe_name = f"{name}_{col}".replace('.', '_').replace('/', '_')
            file_path = os.path.join(output_dir, f"{safe_name}.png")
            plt.savefig(file_path, dpi=300)
            plt.close()

            # --- Create JSON summary ---
            summary_data = {
                "table": name,
                "column": col,
                "total_records": int(ts.sum()),
                "unique_days": int(len(ts)),
                "first_date": str(ts.index.min()),
                "last_date": str(ts.index.max()),
                "average_records_per_day": round(ts.mean(), 2),
                "peak_day": str(ts.idxmax()),
                "peak_count": int(ts.max())
            }

            json_path = os.path.join(output_dir, f"{safe_name}.json")
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(summary_data, f, indent=4, ensure_ascii=False)

categorical charts

In [44]:
output_dir = "categorical_charts"
os.makedirs(output_dir, exist_ok=True)

for name, df in tables_dict.items():
    cat_cols = df.select_dtypes(include=['object', 'category']).columns

    for col in cat_cols:
        vc = df[col].value_counts().head(10)

        if len(vc) <= 1:
            continue

        plt.figure(figsize=(10, 4))
        vc.plot(kind='bar', color='steelblue', edgecolor='black')

        plt.title(f"Top 10 Values — {name}.{col}", fontsize=14, fontweight='bold')
        plt.xlabel(col, fontsize=12)
        plt.ylabel("Count", fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.4)
        plt.tight_layout()

        safe_name = f"{name}_{col}".replace('.', '_').replace('/', '_')
        file_path = os.path.join(output_dir, f"{safe_name}.png")

        plt.savefig(file_path, dpi=300)
        plt.close()

C:\Users\TinTin\AppData\Local\Temp\ipykernel_2164\3747194325.py:21: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
C:\Users\TinTin\AppData\Local\Temp\ipykernel_2164\3747194325.py:21: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
C:\Users\TinTin\AppData\Local\Temp\ipykernel_2164\3747194325.py:21: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
C:\Users\TinTin\AppData\Local\Temp\ipykernel_2164\3747194325.py:21: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
C:\Users\TinTin\AppData\Local\Temp\ipykernel_2164\3747194325.py:21: UserWarning: Tight layout not applied. The left and 

activity trends

In [ ]:
output_dir = "activity_trends"
os.makedirs(output_dir, exist_ok=True)

for name, df in tables_dict.items():
    time_cols = [c for c in df.columns if "created_at" in c or "datetime" in c]

    for col in time_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        ts = df[col].dropna().dt.to_period("M").value_counts().sort_index()

        if len(ts) > 0:
            ts.index = ts.index.to_timestamp()


            plt.figure(figsize=(12, 6))
            ts.plot(kind='line', marker='o', linewidth=2, alpha=0.7)
            plt.title(f"Monthly Activity Trend — {name}.{col}", fontsize=16, fontweight="bold")
            plt.xlabel("Month", fontsize=12)
            plt.ylabel("Record Count", fontsize=12)
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.tight_layout()

            safe_name = f"{name}_{col}".replace('.', '_').replace('/', '_')
            file_path = os.path.join(output_dir, f"{safe_name}.png")
            plt.savefig(file_path, dpi=300)
            plt.show() 

In [46]:
output_dir = "eda_outputs"
os.makedirs(output_dir, exist_ok=True)
for name, df in tables_dict.items():
    df.describe(include='all').to_csv(f"{output_dir}/{name}_describe.csv")
